## Calculating BS Greeks with Python

<b> YouTube Tutorial </b> (Published: Jun 28, 2021):
https://youtu.be/558k7D2alxM


We will be using our code from the Implementation of Black-Scholes formula in Python video.

In [2]:
!pip install py_vollib

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Obtaining dependency information for simplejson from https://files.pythonhosted.org/packages/c3/58/fea732e48a7540035fe46d39e6fd77679f5810311d31da8661ce7a18210a/simplejson-3.19.3-cp311-cp311-macosx_11_0_arm64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.6/74.6 kB 3.9 MB/s eta 0:00:00
  Created wheel for py_vollib: filename=py_vollib-1.0.1-py3-none-any.whl size=62829 sha256=5fbfccae7749a838c951c209bd754093881ad8546fda8cd1a9b8280881929b16
  Stored in directory: /Users/emanueleelias/Library/Caches/pip/wheels/c0/5c/8f/1ed590a10a2cca3cbfa7a7caa29fb5d729b3e1f819bfda4b5e
  Created wheel for py_lets_be_rational: filename=py_lets_be_rational-1.0.1-py3-none-any.whl size=24453 sha256=049a235e09b3f318e7b8774987d7fd710a863ce62a95bc6b7ebb44cf6d4a10c8
  Stored in directory: /Users/emanueleelias/Library/Caches/pip/wheels/d9/20/b1/018f644bacb669d5cd6af600fe09ee0729ae5d75875a9f3043
Successfully built py_vol

In [3]:
from py_vollib.black_scholes import black_scholes as bs
from py_vollib.black_scholes.greeks.analytical import delta, gamma, vega, theta, rho

/Users/emanueleelias/anaconda3/lib/python3.11/site-packages/py_lets_be_rational/numba_helper.py:10: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  return jit(*jit_args, **jit_kwargs)(fun)


In [4]:
# Implementation of Black-Scholes formula in Python
import numpy as np
from scipy.stats import norm
from py_vollib.black_scholes import black_scholes as bs
from py_vollib.black_scholes.greeks.analytical import delta, gamma, vega, theta, rho

# Define variables
r = 0.01       # Tasso di interesse annuale senza rischio (1%)
S = 30         # Prezzo corrente dell'azione sottostante
K = 40         # Prezzo di esercizio dell'opzione
T = 240/365    # Tempo alla scadenza in anni (240 giorni)
sigma = 0.30   # Volatilità implicita (30%)

def blackScholes(r, S, K, T, sigma, type="c"): #ATTENZIONE: valido solo per azioni eruopee
    "Calculate BS price of call/put"
    d1 = (np.log(S/K) + (r + sigma**2/2)*T)/(sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)
    try:
        if type == "c":
            price = S*norm.cdf(d1, 0, 1) - K*np.exp(-r*T)*norm.cdf(d2, 0, 1)
        elif type == "p":
            price = K*np.exp(-r*T)*norm.cdf(-d2, 0, 1) - S*norm.cdf(-d1, 0, 1)
        return price, bs(type, S, K, T, r, sigma)
    except:
        print("Please confirm option type, either 'c' for Call or 'p' for Put!")

In [6]:
print("Option Price: ", blackScholes(r, S, K, T, sigma, "c"))

Option Price:  (0.5132843798399405, 0.5132843798399411)


Perché c'è una discrepanza tra i due valori?

Precisione numerica: I due approcci potrebbero differire nel modo in cui gestiscono gli arrotondamenti.

Algoritmi leggermente diversi: Anche se entrambe le implementazioni seguono il modello di Black-Scholes, le librerie spesso includono ottimizzazioni e possono usare costanti predefinite o tecniche di calcolo diverse.

### Delta
Delta measures the rate of change of the theoretical option value with respect to changes in the underlying asset's price.

$\Delta = \frac{\partial V}{\partial S}$

$\Delta_{call} = \Phi(d1)$

$\Delta_{put} = -\Phi(-d1)$

In [7]:
def delta_calc(r, S, K, T, sigma, type="c"):
    "Calculate delta of an option"
    d1 = (np.log(S/K) + (r + sigma**2/2)*T)/(sigma*np.sqrt(T))
    try:
        if type == "c":
            delta_calc = norm.cdf(d1, 0, 1)
        elif type == "p":
            delta_calc = -norm.cdf(-d1, 0, 1)
        return delta_calc, delta(type, S, K, T, r, sigma)
    except:
        print("Please confirm option type, either 'c' for Call or 'p' for Put!")

### Gamma
Gamma measures the rate of change in the delta with respect to changes in the underlying price.

$\Gamma = \frac{\partial \Delta}{\partial S} = \frac{\partial^2 V}{\partial S^2}$

$\Gamma = \frac{\phi(d1)}{S\sigma\sqrt{\tau}}$

In [8]:
def gamma_calc(r, S, K, T, sigma, type="c"):
    "Calculate gamma of a option"
    d1 = (np.log(S/K) + (r + sigma**2/2)*T)/(sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)
    try:
        gamma_calc = norm.pdf(d1, 0, 1)/(S*sigma*np.sqrt(T))
        return gamma_calc, gamma(type, S, K, T, r, sigma)
    except:
        print("Please confirm option type, either 'c' for Call or 'p' for Put!")

### Vega
Vega measures sensitivity to volatility. Vega is the derivative of the option value with respect to the volatility of the underlying asset.

$\upsilon = \frac{\partial V}{\partial \sigma}$

$\upsilon = S\phi(d1)\sqrt{\tau}$

In [9]:
def vega_calc(r, S, K, T, sigma, type="c"):
    "Calculate BS price of call/put"
    d1 = (np.log(S/K) + (r + sigma**2/2)*T)/(sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)
    try:
        vega_calc = S*norm.pdf(d1, 0, 1)*np.sqrt(T)
        return vega_calc*0.01, vega(type, S, K, T, r, sigma)
    except:
        print("Please confirm option type, either 'c' for Call or 'p' for Put!")

### Theta
Theta measures the sensitivity of the value of the derivative to the passage of time - time decay.

$\Theta = -\frac{\partial V}{\partial \tau}$

$\Theta_{call} = -\frac{S\phi(d1)\sigma}{2\tau} - rK\exp{(-rT)}\Phi(d2)$

$\Theta_{put} = -\frac{S\phi(d1)\sigma}{2\tau} + rK\exp{(-rT)}\Phi(-d2)$

In [10]:
def theta_calc(r, S, K, T, sigma, type="c"):
    "Calculate BS price of call/put"
    d1 = (np.log(S/K) + (r + sigma**2/2)*T)/(sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)
    try:
        if type == "c":
            theta_calc = -S*norm.pdf(d1, 0, 1)*sigma/(2*np.sqrt(T)) - r*K*np.exp(-r*T)*norm.cdf(d2, 0, 1)
        elif type == "p":
            theta_calc = -S*norm.pdf(d1, 0, 1)*sigma/(2*np.sqrt(T)) + r*K*np.exp(-r*T)*norm.cdf(-d2, 0, 1)
        return theta_calc/365, theta(type, S, K, T, r, sigma)
    except:
        print("Please confirm option type, either 'c' for Call or 'p' for Put!")

### Rho
Rho measures the sensitivity to the interest rate.

$\rho = \frac{\partial V}{\partial r}$

$\rho_{call} = K\tau\exp{(-rT)}\Phi(d2)$

$\rho_{put} = -K\tau\exp{(-rT)}\Phi(-d2)$

In [11]:
def rho_calc(r, S, K, T, sigma, type="c"):
    "Calculate BS price of call/put"
    d1 = (np.log(S/K) + (r + sigma**2/2)*T)/(sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)
    try:
        if type == "c":
            rho_calc = K*T*np.exp(-r*T)*norm.cdf(d2, 0, 1)
        elif type == "p":
            rho_calc = -K*T*np.exp(-r*T)*norm.cdf(-d2, 0, 1)
        return rho_calc*0.01, rho(type, S, K, T, r, sigma)
    except:
        print("Please confirm option type, either 'c' for Call or 'p' for Put!")

### All together

In [12]:
option_type='p'

print("Option Price: ", [round(x,3) for x in blackScholes(r, S, K, T, sigma, option_type)])
print("       Delta: ", [round(x,3) for x in delta_calc(r, S, K, T, sigma, option_type)])
print("       Gamma: ", [round(x,3) for x in gamma_calc(r, S, K, T, sigma, option_type)])
print("       Vega : ", [round(x,3) for x in vega_calc(r, S, K, T, sigma, option_type)])
print("       Theta: ", [round(x,3) for x in theta_calc(r, S, K, T, sigma, option_type)])
print("       Rho  : ", [round(x,3) for x in rho_calc(r, S, K, T, sigma, option_type)])

Option Price:  [10.251, 10.251]
       Delta:  [-0.849, -0.849]
       Gamma:  [0.032, 0.032]
       Vega :  [0.057, 0.057]
       Theta:  [-0.003, -0.003]
       Rho  :  [-0.235, -0.235]


Nel tuo caso, l'opzione vale 10.251. Questo è l'importo che un investitore dovrebbe pagare per acquistare questa opzione.

Interpretazione: Un delta di -0.849 significa che, se il prezzo del sottostante aumenta di 1 unità, il prezzo dell'opzione diminuisce di 0.849 unità. Il valore negativo indica che questa è un'opzione put, che si muove inversamente rispetto al prezzo del sottostante.

Interpretazione: Un gamma di 0.032 significa che, se il prezzo del sottostante cambia di 1 unità, il delta dell'opzione cambierà di 0.032.
Gamma è particolarmente importante per valutare la stabilità di Delta e quindi per la gestione del rischio delle posizioni in opzioni.

Interpretazione: Un vega di 0.057 significa che, se la volatilità implicita aumenta di 1 punto percentuale (0.01 in termini decimali), il prezzo dell'opzione aumenterà di 0.057 unità.
Un valore positivo indica che un aumento della volatilità aumenta il valore dell'opzione, poiché maggiore volatilità implica maggiori probabilità di payoff.

Interpretazione: Un theta di -0.003 significa che, con il passare di un giorno, il prezzo dell'opzione diminuisce di 0.003 unità.
Il valore negativo riflette la decadimento temporale, cioè il fatto che il valore dell'opzione diminuisce man mano che si avvicina alla scadenza, poiché il tempo residuo per raggiungere il payoff si riduce.

Interpretazione: Un rho di -0.235 significa che, se il tasso di interesse aumenta dell'1% (0.01 in termini decimali), il prezzo dell'opzione diminuirà di 0.235 unità.
Per le opzioni put, rho è tipicamente negativo: un aumento dei tassi riduce il valore del put, perché rende meno vantaggioso esercitarlo rispetto all'alternativa di investire al tasso di interesse.